# **STEP 10:** Convert RT Dose to NifTI

In [51]:
#!pip install platipy
!pip install unidecode

In [115]:
#the python debugger is a library that provides an interactive source code debugger for Python programs
import pdb

import pydicom
import numpy as np
import nibabel as nib
import SimpleITK as sitk
import os
import shutil
from platipy.dicom.io.rtdose_to_nifti import convert_rtdose
import pandas as pd
import pprint

import re
import unidecode

In [94]:
def duplicate_folder(source_folder, destination_folder):
    # Check if the source folder exists
    if not os.path.exists(source_folder):
        print(f"The source folder '{source_folder}' does not exist.")
        return
    # Check if the destination folder already exists
    if os.path.exists(destination_folder):
        print(f"The destination folder '{destination_folder}' already exists.")
        return
    # Copy the folder
    shutil.copytree(source_folder, destination_folder)
    print(f"Folder '{source_folder}' has been duplicated to '{destination_folder}'.")
# Example usage
source = '/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI'
destination = '/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/'
duplicate_folder(source, destination)

Folder '/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI' has been duplicated to '/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/'.


In [4]:
excluded_patients = ["Patient_53", "Patient_64", "Patient_79", "Patient_92"]

### Organizar os pacientes para lidar com as metástases de cada um

In [95]:
root_dir = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/"

In [96]:
## Colocar só a pasta que queremos para não haver confusões no código

In [97]:
for patient_dir in os.listdir(root_dir):
    #print(f"Processing patient: {patient_dir}")
    patient_id = patient_dir
    patient_path = os.path.join(root_dir, patient_dir)

    for subfolder in os.listdir(patient_path):
        subfolder_path = os.path.join(patient_path, subfolder)
        #print(f"Processing patient: {subfolder_path}")
        if "Fusions" in subfolder or "_RM_" in subfolder: 
            shutil.rmtree(subfolder_path)  # Use rmtree to remove a directory
            print(f"Removed folder: {subfolder_path}")

Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_61/Fusions
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_61/P61_RM_1_TRATAMENTO
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_61/P61_RM_1_FOLLOW_UP
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_78/Fusions
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_78/P78_RM_1_TRATAMENTO
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_78/P78_RM_1_FOLLOW_UP
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_94/Fusions
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_94/P94_RM_1_FOLLOW_UP
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_94/P94_RM_1_TRATAMENTO
Removed folder: /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_53/P53_RM_1_

In [12]:
patient_masks = {}

for patient_dir in os.listdir(root_dir):
    #print(f"Processing patient: {patient_dir}")
    patient_id = patient_dir
    patient_path = os.path.join(root_dir, patient_dir)

    ptv_exists = False #check for the presence of "_PTV" files

    for subfolder in os.listdir(patient_path):
        subfolder_path = os.path.join(patient_path, subfolder)
        #print(f"Processing patient: {subfolder_path}")
        
        for file_name in os.listdir(subfolder_path):
            #print(f"Processing patient: {file_name}")
            if "PTV" in file_name: 
                print("The patient was treated with LINAC")
                ptv_exists = True
                if patient_id not in patient_masks:
                    patient_masks[patient_id] = []
                patient_masks[patient_id].append(file_name)
                
    # After checking for "_PTV", check for "_GTV" and other files - it was adapted according the data I had 
    # it needs to be changed in case in the RTPLAN there are other parts of the brain mentioned and here are not
    for subfolder in os.listdir(patient_path):
        subfolder_path = os.path.join(patient_path, subfolder)
        for file_name in os.listdir(subfolder_path):
            if "GTV" in file_name:
                if not ptv_exists:  # Only add if no "_PTV" files exist
                    print("The patient was treated with Gamma Knife")
                    if patient_id not in patient_masks:
                        patient_masks[patient_id] = []
                    patient_masks[patient_id].append(file_name)
            elif "PTV" not in file_name and "GTV" not in file_name and (file_name.endswith("Brain") or file_name.endswith("Tumor") or file_name.endswith("LEITO") or ("Lesão" in file_name or "Lesao" in file_name)):
                if patient_id not in patient_masks:
                    patient_masks[patient_id] = []
                patient_masks[patient_id].append(file_name)
            elif "PTV" not in file_name and "GTV" not in file_name and ("Met" in file_name or "MtFrontEProf" in file_name):
                if patient_id not in patient_masks:
                    patient_masks[patient_id] = []
                patient_masks[patient_id].append(file_name)
            elif "PTV" not in file_name and "GTV" not in file_name and ("Parasagital" in file_name or "Frontal" in file_name or "Capsular" in file_name or "Fossa" in file_name or "Vermix" in file_name or "Occipital" in file_name or "occipital" in file_name or "Ponte" in file_name or "Parietal" in file_name or "PARASAG" in file_name or "FRONTAL" in file_name or "OCIPTAL" in file_name or "TEMPORAL" in file_name or "frontal" in file_name or "parietal" in file_name):
                if patient_id not in patient_masks:
                    patient_masks[patient_id] = []
                patient_masks[patient_id].append(file_name)
            elif "PTV" not in file_name and "GTV" not in file_name and ("mask_1" in file_name or "mask_2" in file_name or "mask_3" in file_name or "mask_7" in file_name or "mask_8" in file_name or "mask_9" in file_name or "mask_10" in file_name or "mask_16" in file_name or "mask_17" in file_name or "mask_18" in file_name or "mask_20" in file_name):
                if patient_id not in patient_masks:
                    patient_masks[patient_id] = []
                patient_masks[patient_id].append(file_name)
print(patient_masks)

The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with Gamma Knife
The patient was treated with LINAC
The patient was treated with LINAC
The patient was treated with LINAC
T

In [55]:
# Convert the dictionary to a DataFrame
data = []
for patient_id, files in patient_masks.items():
    for file_name in files:
        data.append({"Patient ID": patient_id, "File Name": file_name})
df = pd.DataFrame(data)
# Save the DataFrame to a CSV file
csv_file_path = '/notebooks/disk2/CSV_Dataset/patient_masks.csv'  # Specify your desired CSV file path
df.to_csv(csv_file_path, index=False)
print(f"Patient masks saved to {csv_file_path}")

Patient masks saved to /notebooks/disk2/CSV_Dataset/patient_masks.csv


### Check for each patient how many dose plan they have - and, in case of two or more, check which metastases the doses are

In [90]:
root = "/notebooks/disk2/RT_Dataset_2/02_Patients_NifTI/"

In [91]:
dose_counts = {}

for patient_dir in os.listdir(root):
    patient_id = patient_dir
    patient_path = os.path.join(root, patient_dir)

    dose_count=0
    #print(f"Processing patient: {patient_dir}")
    for subfolder in os.listdir(patient_path):
        subfolder_path = os.path.join(patient_path, subfolder)
        if "CT" in subfolder:
            for file_name in os.listdir(subfolder_path):
                #print(file_name)
                #change if there is another way documented for the dose
                if "RTDOSE" in file_name or "DCMTPS_" in file_name:
                    dose_count += 1
                    if dose_count ==1:
                        dose_counts[patient_id] = dose_count
print(dose_counts)

#for patient_id, count in dose_counts.items():
    #print(f"{patient_id}: {count} dose plan(s) found.")



{'Patient_61': 1, 'Patient_78': 1, 'Patient_94': 1, 'Patient_53': 1, 'Patient_69': 1, 'Patient_89': 1, 'Patient_71': 1, 'Patient_73': 1, 'Patient_90': 1, 'Patient_93': 1, 'Patient_56': 1, 'Patient_75': 1, 'Patient_84': 1, 'Patient_88': 1, 'Patient_76': 1, 'Patient_70': 1, 'Patient_67': 1, 'Patient_80': 1, 'Patient_72': 1, 'Patient_82': 1, 'Patient_96': 1, 'Patient_77': 1, 'Patient_66': 1, 'Patient_58': 1, 'Patient_65': 1, 'Patient_64': 1, 'Patient_86': 1, 'Patient_55': 1, 'Patient_68': 1, 'Patient_63': 1, 'Patient_59': 1, 'Patient_92': 1, 'Patient_79': 1, 'Patient_87': 1, 'Patient_57': 1, 'Patient_81': 1, 'Patient_83': 1, 'Patient_91': 1, 'Patient_54': 1, 'Patient_60': 1, 'Patient_74': 1, 'Patient_95': 1, 'Patient_97': 1, 'Patient_62': 1, 'Patient_85': 1}


In [130]:
def get_metastases(patient_id, patient_masks):
    if patient_id in patient_masks:
        masks = patient_masks[patient_id]
        print(masks)
    else:
        return []  # Return an empty list if the patient ID does not exist
# Example usage
patient_id = 'Patient_97'
metastases = get_metastases(patient_id, patient_masks)

['mask_GTV-insula-E.nii.gz', 'mask_GTV-CEREBELO-D.nii.gz', 'mask_GTV-9.nii.gz', 'mask_GTV-13.nii.gz', 'mask_GTV-8.nii.gz', 'mask_GTV-10.nii.gz', 'mask_GTV-11.nii.gz', 'mask_GTV-6.nii.gz', 'mask_GTV-14.nii.gz', 'mask_GTV-15.nii.gz', 'mask_GTV-pedunculo-cerebelar-D.nii.gz', 'mask_GTV-12.nii.gz', 'mask_GTV-4.nii.gz', 'mask_GTV-16.nii.gz', 'mask_GTV-7.nii.gz', 'mask_GTV-5.nii.gz']


### Now associate the RTDose to the metastases

In [ ]:
### código ainda bastante dependente do utilizador  - patient 64 desconsiderado

In [116]:
#with two or more dose documents

patient_masks_dose_more = {
    'Patient_54': ['mask_PTV-frontal-D-17Gy.nii.gz', 'mask_PTV-frontal-E-post-18Gy.nii.gz', 'mask_PTV-frontal-E-anterior.nii.gz', 'mask_PTV-occiptal-E-18GY.nii.gz'],
    'Patient_59': ['mask_PTVs-1-e-4.nii.gz', 'mask_PTV-3.nii.gz', 'mask_PTV-4.nii.gz', 'mask_PTV-1.nii.gz', 'mask_PTV-2.nii.gz', 'mask_PTVs-2-e-3.nii.gz', 'mask_PTV-2.nii.gz'],
    'Patient_67': ['mask_PTV-18Gy-eval.nii.gz', 'mask_PTV-frontal-E-5x6.nii.gz', 'mask_PTV-frontal-D-18Gy.nii.gz', 'mask_PTV-front-D-p-5x6.nii.gz', 'mask_PTV-temp-E-5x6.nii.gz', 'mask_PTV-esq.nii.gz', 'mask_PTV-parietal-E-5x6.nii.gz', 'mask_PTV-frontal-D-eval.nii.gz', 'mask_PTV-soma-5x6-ok.nii.gz'],
    'Patient_68': ['mask_PTV-frontal-D-3x9.nii.gz', 'mask_PTVFrontais.nii.gz', 'mask_PTV_Pre-Central-Dir-3x9.nii.gz', 'mask_PTV_Pre-Cuneo-_E-3x9.nii.gz', 'mask_PTV-frontal-inf-D-3x9.nii.gz', 'mask_PTV_2-frontais_central.nii.gz', 'mask_PTV_Occiptal_D-3x9.nii.gz', 'mask_PTVPreCentralFrontais.nii.gz', 'mask_PTV_1-cuneooccip.nii.gz', 'mask_PTV_soma.nii.gz', 'mask_GTv-frontal-inf-D.nii.gz'],
    'Patient_69': ['mask_PTV-leito-5x6Gy.nii.gz', 'mask_PTV-temporal-1X20GY.nii.gz'],
    'Patient_70': ['mask_PTV-frontal-D-5x6Gy.nii.gz', 'mask_PTV-temporal-d-5x5Gy.nii.gz', 'mask_cavidade-frontal.nii.gz'],
    'Patient_82': ['mask_PTV-occipital-D3x9Gy.nii.gz', 'mask_PTV-pedunculo-E-3x9Gy-Bra.nii.gz', 'mask_PTV-SOMA.nii.gz'],
    'Patient_85': ['mask_PTV-temp-occiptal-E-3x9.nii.gz', 'mask_PTV-occiptal-D.nii.gz', 'mask_PTV-soma-3x9.nii.gz', 'mask_PTV-frontal-E.nii.gz', 'mask_PTV-3x9.nii.gz'],
    'Patient_88': ['mask_PTV-paracentral-D-20Gy.nii.gz', 'mask_PTV-pos-central-D-20Gy.nii.gz'],
    'Patient_89': ['mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz', 'mask_PTV-soma-3x9.nii.gz', 'mask_PTV-TEMPORAL-ESQ-18GY.nii.gz', 'mask_PTV-FRONTAL-E-18GY.nii.gz', 'mask_PTV-LEITO-3X9GY.nii.gz', 'mask_casca-frontal.nii.gz'],
}

#just with one dose document

patient_masks_dose_single = {
    'Patient_55': ['mask_PTV1-PTV-tumor-auto.nii.gz', 'mask_PTV4-PTV-tumor-auto.nii.gz', 'mask_PTV3-PTV-tumor-auto.nii.gz', 'mask_PTV2-PTV-tumor-auto.nii.gz', 'mask_PTV5-PTV-tumor-auto.nii.gz'],
    'Patient_56': ['mask_2-GTV-met-cerebelo.nii.gz', 'mask_1-GTV-Met.nii.gz', 'mask_2-GTV-met-cerebelo-10-mm.nii.gz', 'mask_1-GTV-Met-10-mm.nii.gz'],
    'Patient_57': ['mask_PTV2-PTV-mts-auto.nii.gz', 'mask_PTV3-PTV-mts-auto.nii.gz', 'mask_PTV5-PTV-mts-auto.nii.gz', 'mask_PTV4-PTV-mts-auto.nii.gz', 'mask_PTV1-PTV-mts-auto.nii.gz'],
    'Patient_58': ['mask_PTV-5x5Gy.nii.gz'],
    'Patient_60': ['mask_PTV-3-x-8Gy.nii.gz', 'mask_PTV-3-x-8Gy-opt.nii.gz'],
    'Patient_61': ['mask_PTV-3x8.nii.gz', 'mask_PTV-3x8-1cm.nii.gz', 'mask_PTV-3x8-opt.nii.gz'],
    'Patient_62': ['mask_PTV-3x9Gy.nii.gz'],
    'Patient_63': ['mask_PTV-3-x-8Gy-1cm.nii.gz', 'mask_PTV-frontal-Esq.nii.gz', 'mask_PTV-ociptal-Esq.nii.gz', 'mask_PTV-peri-ventricual-D.nii.gz', 'mask_PTV-vermis.nii.gz', 'mask_PTVs-3-x-9Gy-1cm.nii.gz'],
    'Patient_65': ['mask_PTV-temporal-E-5x6.nii.gz', 'mask_PTV-parietal-E-5x6Gy.nii.gz', 'mask_PTV-unico-5x6Gy.nii.gz'],
    'Patient_66': ['mask_PTV-1x18.nii.gz'],
    'Patient_71': ['mask_PTV-5x6Gy.nii.gz'],
    'Patient_72': ['mask_GTV-parietal-dir.nii.gz', 'mask_GTV-parietal-dir-2.nii.gz', 'mask_GTV-15.nii.gz', 'mask_GTV-occiptal-esq.nii.gz', 'mask_GTV-frontal-dir.nii.gz', 'mask_GTV-occiptal-dir.nii.gz', 'mask_GTV-cerebelo-dir.nii.gz', 'mask_GTV-occiptal-esq-2.nii.gz'],
    'Patient_73': ['mask_Capsular-Post-D-RM.nii.gz', 'mask_Parasagital-D-RM.nii.gz', 'mask_Fossa-Post-D-RM.nii.gz', 'mask_Occipital-D-RM.nii.gz', 'mask_Vermix-Cereb-D.nii.gz', 'mask_Ponte-D-RM.nii.gz', 'mask_Parasagital-D.nii.gz', 'mask_Ponte-D.nii.gz', 'mask_Parietal-D-RM.nii.gz', 'mask_occipital-D.nii.gz', 'mask_Frontal-E-RM.nii.gz', 'mask_Capsular-Post-D.nii.gz', 'mask_Fossa-Post-D.nii.gz', 'mask_Frontal-E.nii.gz', 'mask_Parietal-D.nii.gz', 'mask_Vermix-Cereb-D-RM.nii.gz'],
    'Patient_74': ['mask_PARASAG-POST-D.nii.gz', 'mask_OCIPTAL-D.nii.gz', 'mask_TEMPORAL-D.nii.gz', 'mask_FRONTAL-E.nii.gz', 'mask_FRONTAL-SUPERIOR-E.nii.gz'],
    'Patient_75': ['mask_GTV2-superior.nii.gz', 'mask_GTV-frontal-D.nii.gz'],
    'Patient_76': ['mask_GTV2.nii.gz', 'mask_GTV1.nii.gz', 'mask_GTV3.nii.gz'],
    'Patient_77': ['mask_14.nii.gz', 'mask_8.nii.gz', 'mask_9.nii.gz', 'mask_13.nii.gz', 'mask_3.nii.gz', 'mask_11.nii.gz', 'mask_16.nii.gz', 'mask_20.nii.gz', 'mask_15.nii.gz', 'mask_10.nii.gz', 'mask_1.nii.gz', 'mask_7.nii.gz', 'mask_2.nii.gz', 'mask_19.nii.gz', 'mask_17.nii.gz', 'mask_18.nii.gz'],
    'Patient_78': ['mask_MtFrontEProf.nii.gz', 'mask_MetVerCerebelar.nii.gz', 'mask_MetFDirInf.nii.gz', 'mask_MetODir.nii.gz', 'mask_MetCapIntDir.nii.gz', 'mask_MetIntHemisf.nii.gz', 'mask_MetFPDIR.nii.gz', 'mask_MetParProfD.nii.gz', 'mask_MetOccEsqInf.nii.gz', 'mask_MetMesencDir.nii.gz', 'mask_MetHCEs.nii.gz', 'mask_MetTempAntE.nii.gz', 'mask_MetOccE2Post.nii.gz', 'mask_MetTempPostE.nii.gz', 'mask_MetParDir1.nii.gz', 'mask_MetHCDir.nii.gz', 'mask_MetODir3.nii.gz', 'mask_MetTempEsq.nii.gz', 'mask_MetOccEs1Ant.nii.gz', 'mask_MetODir2.nii.gz', 'mask_MetOccDir2.nii.gz', 'mask_MetODir1.nii.gz', 'mask_MetFEsq.nii.gz', 'mask_MetaTpDir.nii.gz', 'mask_MetaTpE.nii.gz', 'mask_MetAGPdirAnt.nii.gz', 'mask_MetParDir2.nii.gz'],
    'Patient_80': ['mask_PTV-18Gy.nii.gz'],
    'Patient_81': ['mask_PTV-FORNTAL-E-20GY.nii.gz'],
    'Patient_83': ['mask_PTV-Lesao-Cerebelar-D-20G.nii.gz', 'mask_PTV-Lesao-Occipitotempor.nii.gz', 'mask_Lesao-Occipitotemporal-Di.nii.gz', 'mask_Lesao-Cerebelar-Dir.nii.gz'],
    'Patient_84': ['mask_PTV-1-x-20Gy.nii.gz'],
    'Patient_86': ['mask_PTV-1x20Gy.nii.gz'],
    'Patient_87': ['mask_PTV_leito_5x6Gy.nii.gz', 'mask_PTV_OcipDir_5x7Gy.nii.gz', 'mask_PTV_leito_EVAL.nii.gz'],
    'Patient_90': ['mask_Lesao-bulbar.nii.gz', 'mask_Lesao-P-D.nii.gz', 'mask_Lesao-talamo-E.nii.gz', 'mask_Lesao-periventricular-D.nii.gz', 'mask_Lesao-paramediana-D.nii.gz', 'mask_Lesao-cuneo-D.nii.gz', 'mask_Lesao-hipocampo-E.nii.gz', 'mask_Lesao-temporal-D.nii.gz', 'mask_Lesao-occipital-D.nii.gz', 'mask_Lesao-talamo-D.nii.gz', 'mask_Lesao-insula-E.nii.gz', 'mask_Lesao-P-D-superior.nii.gz', 'mask_Lesao-P-E.nii.gz'],
    'Patient_91': ['mask_GTV-parietal-post.nii.gz', 'mask_GTV-cerebelo-E.nii.gz', 'mask_GTV-occiptal-E-2.nii.gz', 'mask_GTV-E-corpo-caloso.nii.gz', 'mask_GTV-occiptal-E.nii.gz', 'mask_GTv-parietal-D.nii.gz'],
    'Patient_93': ['mask_frontal-E.nii.gz', 'mask_frontal-D.nii.gz', 'mask_parietal-E.nii.gz'],
    'Patient_94': ['mask_3.nii.gz', 'mask_1.nii.gz', 'mask_2.nii.gz'],
    'Patient_95': ['mask_GTV-fronto-parietal-D.nii.gz', 'mask_GTV-temporal-E.nii.gz', 'mask_GTV-temporal-d.nii.gz'],
    'Patient_96': ['mask_GTV-PARIETAL-D.nii.gz', 'mask_FRONTAL-D.nii.gz'],
    'Patient_97': ['mask_GTV-insula-E.nii.gz', 'mask_GTV-CEREBELO-D.nii.gz', 'mask_GTV-9.nii.gz', 'mask_GTV-13.nii.gz', 'mask_GTV-8.nii.gz', 'mask_GTV-10.nii.gz', 'mask_GTV-11.nii.gz', 'mask_GTV-6.nii.gz', 'mask_GTV-14.nii.gz', 'mask_GTV-15.nii.gz', 'mask_GTV-pedunculo-cerebelar-D.nii.gz', 'mask_GTV-12.nii.gz', 'mask_GTV-4.nii.gz', 'mask_GTV-16.nii.gz', 'mask_GTV-7.nii.gz', 'mask_GTV-5.nii.gz']
}

In [117]:
root_dir = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/"
root_dir_1 = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/"

### One Dose Document

In [9]:
def get_sop_instance_uid(dicom_data):
    return (dicom_data.get((0x0008, 0x0018), "")).value  # SOP Instance UID

def get_referenced_rt_plan_sop_uid(dicom_data):
    # Reads the Referenced RT Plan Sequence from RTDOSE file
    seq = dicom_data.get((0x300c, 0x0002))
    if seq:
        ref_sop = seq[0].get((0x0008, 0x1155))  # Referenced SOP Instance UID
        if ref_sop:
            return ref_sop.value

def get_dose_reference_description(dicom_data):
    seq = dicom_data.get((0x300a, 0x0010))  # Dose Reference Sequence in RTPLAN
    if seq:
        for item in seq:
            desc = item.get((0x300a, 0x0016))  # Dose Reference Description
            if desc:
                return desc.value

def dose(root_dir, patient_masks):
    files = {}

    for patient_dir in os.listdir(root_dir):
        patient_id = patient_dir
        patient_path = os.path.join(root_dir, patient_dir)
        files[patient_id] = {}

        rtplan_map = {}  # SOP Instance UID -> Dose Description
        rtdose_map = {}  # RTDOSE file -> Referenced RTPLAN SOP UID

        for subfolder in os.listdir(patient_path):
            subfolder_path = os.path.join(patient_path, subfolder)
            if "CT" in subfolder:
                for file_name in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, file_name)
                    file_name_lower = file_name.lower()

                    #print("Checking file:", file_name)
                    if "rtplan" in file_name.lower() or "dcmrt_plan" in file_name.lower():
                        try:
                            dcm = pydicom.dcmread(file_path, force=True)
                            sop_uid_plan = get_sop_instance_uid(dcm)
                            dose_desc = get_dose_reference_description(dcm)
                            rtplan_map[sop_uid_plan] = dose_desc
                            #print("RTPLAN map:", rtplan_map)
                        except Exception as e:
                            print(f"Error reading RTPLAN {file_path}: {e}")
    
                    if "rtdose" in file_name_lower or "dcmtps_" in file_name_lower:
                        try:
                            dcm = pydicom.dcmread(file_path, force=True)
                            ref_sop_uid = get_referenced_rt_plan_sop_uid(dcm)
                            rtdose_map[file_name] = ref_sop_uid
                            #print("RTDOSE map:", rtdose_map)
                        except Exception as e:
                            print(f"Error reading RTDOSE {file_path}: {e}")    
                 
        metastases = patient_masks.get(patient_id, [])
        #print(f"Metastases for patient {patient_id}: {metastases}") 
        for dose_file, ref_sop_uid in rtdose_map.items():
            #print(f"Processing dose file: {dose_file} with linked SOP UID: {ref_sop_uid}")
            #print(f"Processing SOP UID from Plan Map: {dose_desc} with linked SOP UID: {sop_uid_plan}")
            if ref_sop_uid == sop_uid_plan:
                files[patient_id][dose_file] = metastases
            
    return files

associated_files = dose(root_dir_1, patient_masks_dose_single)

for patient_id, associations in associated_files.items():
    print(f"Patient: {patient_id}")
    for dose_file, mets in associations.items():
        print(f"Dose File: {dose_file} -> Metastases: {mets}")
        print("------")


Patient: Patient_61
Dose File: DCMTPS_Calculated.dcm -> Metastases: ['mask_PTV-3x8.nii.gz', 'mask_PTV-3x8-1cm.nii.gz', 'mask_PTV-3x8-opt.nii.gz']
------
Patient: Patient_78
Dose File: RTDOSE.dcm -> Metastases: ['mask_MtFrontEProf.nii.gz', 'mask_MetVerCerebelar.nii.gz', 'mask_MetFDirInf.nii.gz', 'mask_MetODir.nii.gz', 'mask_MetCapIntDir.nii.gz', 'mask_MetIntHemisf.nii.gz', 'mask_MetFPDIR.nii.gz', 'mask_MetParProfD.nii.gz', 'mask_MetOccEsqInf.nii.gz', 'mask_MetMesencDir.nii.gz', 'mask_MetHCEs.nii.gz', 'mask_MetTempAntE.nii.gz', 'mask_MetOccE2Post.nii.gz', 'mask_MetTempPostE.nii.gz', 'mask_MetParDir1.nii.gz', 'mask_MetHCDir.nii.gz', 'mask_MetODir3.nii.gz', 'mask_MetTempEsq.nii.gz', 'mask_MetOccEs1Ant.nii.gz', 'mask_MetODir2.nii.gz', 'mask_MetOccDir2.nii.gz', 'mask_MetODir1.nii.gz', 'mask_MetFEsq.nii.gz', 'mask_MetaTpDir.nii.gz', 'mask_MetaTpE.nii.gz', 'mask_MetAGPdirAnt.nii.gz', 'mask_MetParDir2.nii.gz']
------
Patient: Patient_94
Dose File: RTDOSE.dcm -> Metastases: ['mask_3.nii.gz', 'ma

In [10]:
files = {
    patient_id: {
        dose_file: mets
        for dose_file, mets in dose_info.items()
        if dose_file != '.ipynb_checkpoints'
    }
    for patient_id, dose_info in associated_files.items()
    if patient_id != '.ipynb_checkpoints'
}

pprint.pprint(files, width=120)

{'Patient_55': {'RTDOSE_P1.dcm': ['mask_PTV1-PTV-tumor-auto.nii.gz',
                                  'mask_PTV4-PTV-tumor-auto.nii.gz',
                                  'mask_PTV3-PTV-tumor-auto.nii.gz',
                                  'mask_PTV2-PTV-tumor-auto.nii.gz',
                                  'mask_PTV5-PTV-tumor-auto.nii.gz']},
 'Patient_56': {'RTDOSE_P2.dcm': ['mask_2-GTV-met-cerebelo.nii.gz',
                                  'mask_1-GTV-Met.nii.gz',
                                  'mask_2-GTV-met-cerebelo-10-mm.nii.gz',
                                  'mask_1-GTV-Met-10-mm.nii.gz']},
 'Patient_57': {'RTDOSE_P3.dcm': ['mask_PTV2-PTV-mts-auto.nii.gz',
                                  'mask_PTV3-PTV-mts-auto.nii.gz',
                                  'mask_PTV5-PTV-mts-auto.nii.gz',
                                  'mask_PTV4-PTV-mts-auto.nii.gz',
                                  'mask_PTV1-PTV-mts-auto.nii.gz']},
 'Patient_58': {'DCMTPS_Calculated.dcm': ['mask_

In [11]:
dose_distribution_files = {}

for patient, documents in files.items():
    print(f"Processing {patient}")
    number = patient.split("_")[1]

    for rt_dose_file in documents:

        dicom_path = f"/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_{number}/P{number}_CT_1_TRATAMENTO/"
        rt_dose_path = os.path.join(dicom_path, rt_dose_file)

        if "DCMTPS_Calculated" in rt_dose_file or "RTDOSE" in rt_dose_file:
            output_volume_path = f"/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_{number}/P{number}_CT_1_TRATAMENTO/P{number}_Dose_Distribution.nii.gz"

        #Convert RTDOSE to NIfTI
        convert_rtdose(dcm_dose=rt_dose_path, force=True, dose_output_path=output_volume_path)
        print(f"RTDOSE file {rt_dose_file} converted to NIfTI")

        # Add to result dictionary
        dose_distribution_files.setdefault(patient, {})[output_volume_path] = documents[rt_dose_file]

    print(f"✅ Done with {patient}\n")
print(dose_distribution_files)

Processing Patient_61
RTDOSE file DCMTPS_Calculated.dcm converted to NIfTI
✅ Done with Patient_61

Processing Patient_78
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_78

Processing Patient_94
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_94

Processing Patient_71
RTDOSE file DCMTPS_Calculated.dcm converted to NIfTI
✅ Done with Patient_71

Processing Patient_73
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_73

Processing Patient_90
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_90

Processing Patient_93
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_93

Processing Patient_56
RTDOSE file RTDOSE_P2.dcm converted to NIfTI
✅ Done with Patient_56

Processing Patient_75
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_75

Processing Patient_84
RTDOSE file DCMTPS_Calculated.dcm converted to NIfTI
✅ Done with Patient_84

Processing Patient_76
RTDOSE file RTDOSE.dcm converted to NIfTI
✅ Done with Patient_

In [9]:
# Tenho que ir buscar a tag 300a x 0026 ao ficheiro do plan, associar à metastase e ao paciente. 
# Depois ver qual é o volume no dicomplyer e também descobrir o raio dependendo se for 25 cm para baixo 1–2 mm se for mais 4-6mm

In [90]:
print(dose_distribution_files)

{'Patient_61': {'/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_61/P61_CT_1_TRATAMENTO/P61_Dose_Distribution.nii.gz': ['mask_PTV-3x8.nii.gz', 'mask_PTV-3x8-1cm.nii.gz', 'mask_PTV-3x8-opt.nii.gz']}, 'Patient_78': {'/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_78/P78_CT_1_TRATAMENTO/P78_Dose_Distribution.nii.gz': ['mask_MtFrontEProf.nii.gz', 'mask_MetVerCerebelar.nii.gz', 'mask_MetFDirInf.nii.gz', 'mask_MetODir.nii.gz', 'mask_MetCapIntDir.nii.gz', 'mask_MetIntHemisf.nii.gz', 'mask_MetFPDIR.nii.gz', 'mask_MetParProfD.nii.gz', 'mask_MetOccEsqInf.nii.gz', 'mask_MetMesencDir.nii.gz', 'mask_MetHCEs.nii.gz', 'mask_MetTempAntE.nii.gz', 'mask_MetOccE2Post.nii.gz', 'mask_MetTempPostE.nii.gz', 'mask_MetParDir1.nii.gz', 'mask_MetHCDir.nii.gz', 'mask_MetODir3.nii.gz', 'mask_MetTempEsq.nii.gz', 'mask_MetOccEs1Ant.nii.gz', 'mask_MetODir2.nii.gz', 'mask_MetOccDir2.nii.gz', 'mask_MetODir1.nii.gz', 'mask_MetFEsq.nii.gz', 'mask_MetaTpDir.nii.gz', 'mask_MetaTpE.nii.gz', 'mask_

In [12]:
df = pd.read_excel("/notebooks/disk2/CSV_Dataset/Volumes_Doses.xlsx")

##### Este código abaixo foi desenvolvido para descobrir os volumes das metástases dos pacientes que não foram possível abrir no dicomplyer - no caso do dicomplyer conseguimos descobrir a dose média administrada a cada um dos pacientes em % - desta forma, vai se tentar encontrar o mesmo através dos ficheiros dose em nifti

In [10]:
import nibabel as nib
import numpy as np

mask_img = nib.load("/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_90/P90_CT_1_TRATAMENTO/mask_Lesao-temporal-D.nii.gz")
mask_data = mask_img.get_fdata()

# Verify how many voxels have > 0
voxel_count = np.sum(mask_data > 0)

# Volume of each voxel
voxel_spacing = mask_img.header.get_zooms()  # (x, y, z) spacing em mm
voxel_volume_mm3 = np.prod(voxel_spacing)    # volume de um voxel em mm³

#Calculating the volume
total_volume_mm3 = voxel_count * voxel_volume_mm3
total_volume_cm3 = total_volume_mm3 / 1000    # converte para cm³

print(f"Nº of voxels = {voxel_count}")
print(f"Total Volume = {total_volume_mm3:.2f} mm³")
print(f"Total Volume = {total_volume_cm3:.2f} cm³")


Nº of voxels = 3922
Total Volume = 1149.02 mm³
Total Volume = 1.15 cm³


In [13]:
def target_doses(dicom_data):
    doses = []
    seq = dicom_data.get((0x300a, 0x0010))  # Dose Reference Sequence
    if seq:
        for item in seq:
            dose = item.get((0x300a, 0x0026))  # Target Prescription Dose
            desc = item.get((0x300a, 0x0016))  # Dose Reference Description
            if dose:
                doses.append((dose.value, desc.value if desc else ""))
    return doses

for patient_dir in os.listdir(root_dir_1):
    patient_id = patient_dir
    patient_path = os.path.join(root_dir_1, patient_dir)

    rtplan_map = {}
    
    for subfolder in os.listdir(patient_path):
        #print(subfolder)
        subfolder_path = os.path.join(patient_path, subfolder)
        if "CT" in subfolder:
            for file_name in os.listdir(subfolder_path):
                file_path = os.path.join(subfolder_path, file_name)
                file_name_lower = file_name.lower()
                #print(file_name_lower)
                if "rtplan" in file_name.lower() or "dcmrt_plan" in file_name.lower():
                    try:
                        dcm = pydicom.dcmread(file_path, force=True)
                        doses = target_doses(dcm)
                        for dose, desc in doses:
                            rtplan_map[desc] = dose
                        
                    except Exception as e:
                        print(f"Error reading RTPLAN {file_path}: {e}")
    #print(rtplan_map)
    ids = ['Patient_56', 'Patient_83', 'Patient_90']
    if patient_id in patient_masks_dose_single:
        for dose, desc in rtplan_map.items():
            print(patient_id)
            print(f"---")
            print(f"  Dose: {dose} Gy, Description: {desc}")


Patient_61
---
  Dose: CRANIO Gy, Description: 24.00
Patient_78
---
  Dose: B:MetAGPdirAnt Gy, Description: 18
Patient_78
---
  Dose: C:MetaTpDir Gy, Description: 18
Patient_78
---
  Dose: E:MetCapIntDir Gy, Description: 16
Patient_78
---
  Dose: H:MetFPDIR Gy, Description: 18
Patient_78
---
  Dose: I:MetHCDir Gy, Description: 18
Patient_78
---
  Dose: J:MetHCEs Gy, Description: 18
Patient_78
---
  Dose: K:MetIntHemisf Gy, Description: 18
Patient_78
---
  Dose: L:MetMesencDir Gy, Description: 14
Patient_78
---
  Dose: M:MetOccDir2 Gy, Description: 18
Patient_78
---
  Dose: N:MetOccE2Post Gy, Description: 18
Patient_78
---
  Dose: O:MetOccEs1Ant Gy, Description: 18
Patient_78
---
  Dose: P:MetOccEsqInf Gy, Description: 18
Patient_78
---
  Dose: Q:MetODir Gy, Description: 18
Patient_78
---
  Dose: R:MetODir1 Gy, Description: 18
Patient_78
---
  Dose: S:MetODir2 Gy, Description: 18
Patient_78
---
  Dose: T:MetODir3 Gy, Description: 18
Patient_78
---
  Dose: V:MetParDir2 Gy, Description: 1

#### Criar as máscaras de isodose e depois ajustá-las às máscaras do volume dos tumores

In [18]:
patients = ["Patient_56", "Patient_83", "Patient_90"]


for patient, data in dose_distribution_files.items():
    print(f"Processing {patient}")
    number = str(patient).replace("Patient_", "").replace("P", "")
    documents = dose_distribution_files[patient]
    if patient in patient_masks_dose_single:
        for dose_distribution_file, masks in documents.items():
            print(dose_distribution_file)
            dose_distribution = nib.load(dose_distribution_file)
            dose_distribution_data = dose_distribution.get_fdata()
            #print(f"Data shape:", dose_distribution_data.shape)
            #print(f"Unique values:", np.unique(dose_distribution_data))
    
            for mask in masks:
                print(f"Working on mask: {mask}")
                mask_name = os.path.basename(mask).replace(".nii.gz", "")
                matched_rows = df[(df["File Name"] == mask_name) & (df["Patient ID"] == patient)]

                if not matched_rows.empty:
                    for _, row in matched_rows.iterrows():
                        if patient in patients: # the dose description from the DICOM files was used
                            prescription = desc 
                            print(f"    Prescription dose for {patient} and {mask}: {prescription}")
                            isodose_volume_mask = (dose_distribution_data >= prescription_dose)
                           
                            
                        else: #description used from the Dicomplyer
                            prescription_dose = (row["Dose Media"] / 100) * desc
                            print(f"    Prescription dose for {patient} and mask {mask_name}: {prescription_dose}")
                            isodose_volume_mask = (dose_distribution_data >= prescription_dose)
          
                        isodose_volume_image = nib.Nifti1Image(isodose_volume_mask, dose_distribution.affine, dose_distribution.header)
                        
                        isodose_volume_path = f"/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_{number}/P{number}_CT_1_TRATAMENTO/P{number}_Isodose_Volume_{mask}"
                        nib.save(isodose_volume_image, isodose_volume_path)
                        print(f" Isodose mask saved for {mask}")
                else:
                    print(f"     No matching dose found for {patient} and mask {mask_name}")
                    

                # Resize Isodose Volume Mask to the same caracteristics as the original mask volume
                path = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_" + number + "/P" + number + "_CT_1_TRATAMENTO/"
                tumor_mask_path = os.path.join(path, mask)
                
                tumor_mask = sitk.ReadImage(tumor_mask_path) # tumor path
                isodose_volume = sitk.ReadImage(isodose_volume_path) # isodose path
        
                resampler = sitk.ResampleImageFilter()
        
                resampler.SetReferenceImage(tumor_mask)
            
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                
                resampler.SetOutputOrigin(tumor_mask.GetOrigin())
                resampler.SetOutputSpacing(tumor_mask.GetSpacing())
                resampler.SetOutputDirection(tumor_mask.GetDirection())
                resampler.SetSize(tumor_mask.GetSize())
                
                resampled_volume = resampler.Execute(isodose_volume)
        
                resampled_volume_path = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_" + number + "/P" + number + "_CT_1_TRATAMENTO/P" + number + "_Isodose_Volume_" + mask_name + "_Resampled.nii.gz"
                sitk.WriteImage(resampled_volume, resampled_volume_path)
                print(f"Isodose Prescription Volume resampled")
    
                # Filter for each mask
                isodose_volume_resized = nib.load(resampled_volume_path)
                isodose_volume_resized_data = isodose_volume_resized.get_fdata()
        
                mask_tumor = nib.load(tumor_mask_path)
                mask_tumor_data = mask_tumor.get_fdata()
                
                coords = np.argwhere(mask_tumor_data > 0)
                centroide = np.mean(coords, axis=0).astype(int)
        
                shape = isodose_volume_resized_data.shape
                x, y, z = np.ogrid[:shape[0], :shape[1], :shape[2]]
                
                dx, dy, dz = mask_tumor.header.get_zooms() #so the distance is in mm
                distance = np.sqrt(
                    ((x - centroide[0]) * dx) ** 2 +
                    ((y - centroide[1]) * dy) ** 2 +
                    ((z - centroide[2]) * dz) ** 2
                )
        
                volume_cm3 = matched_rows.iloc[0]["Volume"]
                volume_mm3 = volume_cm3 * 1000
                r = (3 * volume_mm3 / (4 * np.pi)) ** (1/3)
                r_new = (3 * 1.50 * volume_mm3 / (4 * np.pi)) ** (1/3) #margem de 50% mais para o número de voxeis ser aceitável
                margin = r_new - r
                min_voxel_size = min(dx, dy, dz)  # em mm
                if margin < min_voxel_size: # isto nos casos em que a margin é menor do que 1, o que significa que temos que garantir que a margem seja pelo menos igual ao tamanho de um voxel
                    print(f"Margin smaller than 1 voxel ({margin:.2f} mm), adjusting to {min_voxel_size:.2f} mm")
                    margin = min_voxel_size
                print(f"Computed margin: {margin:.2f} mm for patient {patient}, mask {mask_name}")

                sphere_mask = distance <= margin
                print(f"Voxels in sphere_mask:", np.count_nonzero(sphere_mask))
        
                filtered_volume = isodose_volume_resized_data * sphere_mask
                final_volume = nib.Nifti1Image(filtered_volume, affine=isodose_volume_resized.affine)
        
                filtered_volume_path = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_" + number + "/P" + number + "_CT_1_TRATAMENTO/P" + number + "_Isodose_Volume_" + mask_name + "_Resampled_Filtered.nii.gz"
                nib.save(final_volume, filtered_volume_path)
                print(f"Isodose Prescription Volume Filtered")
                print()
     

Processing Patient_61
/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose/Patient_61/P61_CT_1_TRATAMENTO/P61_Dose_Distribution.nii.gz
Working on mask: mask_PTV-3x8.nii.gz
    Prescription dose for Patient_61 and mask mask_PTV-3x8: 28.351349999999996
 Isodose mask saved for mask_PTV-3x8.nii.gz
Isodose Prescription Volume resampled
Computed margin: 1.75 mm for patient Patient_61, mask mask_PTV-3x8
Voxels in sphere_mask: 27
Isodose Prescription Volume Filtered

Working on mask: mask_PTV-3x8-1cm.nii.gz
    Prescription dose for Patient_61 and mask mask_PTV-3x8-1cm: 19.625220000000002
 Isodose mask saved for mask_PTV-3x8-1cm.nii.gz
Isodose Prescription Volume resampled
Computed margin: 3.29 mm for patient Patient_61, mask mask_PTV-3x8-1cm
Voxels in sphere_mask: 171
Isodose Prescription Volume Filtered

Working on mask: mask_PTV-3x8-opt.nii.gz
    Prescription dose for Patient_61 and mask mask_PTV-3x8-opt: 27.944729999999996
 Isodose mask saved for mask_PTV-3x8-opt.nii.gz
Isodose Prescripti

### For more than one Dose Document

##### Problema nestes casos - os ficheiros DICOM destes pacientes não tem todos os parâmetros que utilizamos, pelo que vamos na mesma utilizar o SOP Instance UID para associar o plano ao documento da dose

Associar as metástases aos planos de forma manual

In [121]:
## import difflib
def target_prescription (dicom_data):
    # Gets the target prescription dose for the RT Plan file
    dose_ref_seq = dicom_data.get((0x300A, 0x0010))  # Dose Reference Sequence
    if dose_ref_seq:
        for item in dose_ref_seq:
            target_dose = item.get((0x300A, 0x0026))  # Target Prescription Dose
            if target_dose:
                return float(target_dose.value)
    return None

def get_rt_name (dicom_data):
    # Gets RT Plan Name 
    seq = dicom_data.get(((0x300a, 0x0003)))
    if seq:
        return seq.value
    return None

def clean_text(text):
    if not text:
        return ""
    text = unidecode.unidecode(text)
    text = text.lower()
    text = re.sub(r'[^a-z0-9]', '', text)
    return text

def dose(root_dir, patient_masks, name_list):
    files = {}

    for patient_dir in os.listdir(root_dir):
        if patient_dir.startswith("."):
            continue

        patient_id = patient_dir
        patient_path = os.path.join(root_dir, patient_dir)
        files[patient_id] = {}

        rtplan_map = {}

        for subfolder in os.listdir(patient_path):
            subfolder_path = os.path.join(patient_path, subfolder)
            if "CT" in subfolder:
                for file_name in os.listdir(subfolder_path):
                    file_path = os.path.join(subfolder_path, file_name)
                    if any(key in file_name.lower() for key in ["rtplan", "dcmrt_plan"]):
                        try:
                            dcm = pydicom.dcmread(file_path, force=True)
                            dose_val = target_prescription(dcm)
                            rt_name = clean_text(get_rt_name(dcm))

                            if rt_name:
                                rtplan_map[rt_name] = {
                                    "dose": dose_val,
                                    "file": file_name
                                }
                        except Exception as e:
                            print(f"❌ Error in {file_path}: {e}")

        metastases = patient_masks.get(patient_id, [])

        print(f"\nPatient: {patient_id}")
        print("RTPLAN descriptions:")
        for desc in rtplan_map:
            print(f"  - {desc}")
        print("Masks:")
        for m in metastases:
            print(f"  - {m}")

        for name in name_list:
            name_clean = clean_text(name)
            for desc, dose_info in rtplan_map.items():
                if name_clean in desc:
                    for mask in metastases:
                        mask_clean = clean_text(mask)
                        if name_clean in mask_clean:
                            print(f"✅ Match: '{name}' in '{desc}' and '{mask}' → {dose_info['dose']} Gy")
                            files[patient_id][f"{name}_TO_{mask}"] = {
                                "dose": dose_info["dose"],
                                "rtplan_file": dose_info["file"],
                                "metastasis": mask
                            }
        matched_masks = set()
        for key in files[patient_id]:
            matched_masks.add(files[patient_id][key]["metastasis"])
        
        for mask in metastases:
            if mask not in matched_masks:
                # Ask user to input dose manually
                while True:
                    user_input = input(f"No match found for mask '{mask}'. Please enter the correct dose (Gy), or leave blank to skip: ").strip()
                    if user_input == "":
                        print(f"Skipping mask '{mask}' (no dose entered).")
                        break
                    try:
                        dose_value = float(user_input)
                        files[patient_id][f"manual_TO_{mask}"] = {
                            "dose": dose_value,
                            "rtplan_file": "manual_input",
                            "metastasis": mask
                        }
                        print(f"Inserted manual dose {dose_value} Gy for mask '{mask}'.")
                        break
                    except ValueError:
                        print("Invalid input, please enter a numeric value for the dose.")


    return files

names = ['temporal', 'leito', 'temporal', 'frontal', 'paracentral', 'poscentral', 'parietal', 'soma', 'occipital', 'frontaisprecentral','precuneooccip', 'occipitalnovo', 'occipital1', 'plano1ptvs', '2e3', 'plano5ptvs', '1e4']
associated_files = dose(root_dir, patient_masks_dose_more, names)
print(associated_files)


Patient: Patient_69
RTPLAN descriptions:
  - temporal
  - leito3
Masks:
  - mask_PTV-leito-5x6Gy.nii.gz
  - mask_PTV-temporal-1X20GY.nii.gz
✅ Match: 'temporal' in 'temporal' and 'mask_PTV-temporal-1X20GY.nii.gz' → 20.0 Gy
✅ Match: 'leito' in 'leito3' and 'mask_PTV-leito-5x6Gy.nii.gz' → 30.0 Gy
✅ Match: 'temporal' in 'temporal' and 'mask_PTV-temporal-1X20GY.nii.gz' → 20.0 Gy

Patient: Patient_89
RTPLAN descriptions:
  - temporal
  - lesoes27gy
  - frontal
Masks:
  - mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz
  - mask_PTV-soma-3x9.nii.gz
  - mask_PTV-TEMPORAL-ESQ-18GY.nii.gz
  - mask_PTV-FRONTAL-E-18GY.nii.gz
  - mask_PTV-LEITO-3X9GY.nii.gz
  - mask_casca-frontal.nii.gz
✅ Match: 'temporal' in 'temporal' and 'mask_PTV-TEMPORAL-ESQ-18GY.nii.gz' → 18.0 Gy
✅ Match: 'temporal' in 'temporal' and 'mask_PTV-TEMPORAL-ESQ-18GY.nii.gz' → 18.0 Gy
✅ Match: 'frontal' in 'frontal' and 'mask_PTV-FRONTAL-E-18GY.nii.gz' → 18.0 Gy
✅ Match: 'frontal' in 'frontal' and 'mask_casca-frontal.nii.gz' → 18.0 Gy


No match found for mask 'mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz'. Please enter the correct dose (Gy), or leave blank to skip:  18


Inserted manual dose 18.0 Gy for mask 'mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz'.


No match found for mask 'mask_PTV-soma-3x9.nii.gz'. Please enter the correct dose (Gy), or leave blank to skip:  27


Inserted manual dose 27.0 Gy for mask 'mask_PTV-soma-3x9.nii.gz'.


...

Inserted manual dose 27.0 Gy for mask 'mask_PTV-occiptal-D.nii.gz'.


No match found for mask 'mask_PTV-soma-3x9.nii.gz'. Please enter the correct dose (Gy), or leave blank to skip:  27


Inserted manual dose 27.0 Gy for mask 'mask_PTV-soma-3x9.nii.gz'.


No match found for mask 'mask_PTV-3x9.nii.gz'. Please enter the correct dose (Gy), or leave blank to skip:  27


Inserted manual dose 27.0 Gy for mask 'mask_PTV-3x9.nii.gz'.
{'Patient_69': {'temporal_TO_mask_PTV-temporal-1X20GY.nii.gz': {'dose': 20.0, 'rtplan_file': 'DCMRT_Plan_1.dcm', 'metastasis': 'mask_PTV-temporal-1X20GY.nii.gz'}, 'leito_TO_mask_PTV-leito-5x6Gy.nii.gz': {'dose': 30.0, 'rtplan_file': 'DCMRT_Plan.dcm', 'metastasis': 'mask_PTV-leito-5x6Gy.nii.gz'}}, 'Patient_89': {'temporal_TO_mask_PTV-TEMPORAL-ESQ-18GY.nii.gz': {'dose': 18.0, 'rtplan_file': 'DCMRT_Plan_1.dcm', 'metastasis': 'mask_PTV-TEMPORAL-ESQ-18GY.nii.gz'}, 'frontal_TO_mask_PTV-FRONTAL-E-18GY.nii.gz': {'dose': 18.0, 'rtplan_file': 'DCMRT_Plan_2.dcm', 'metastasis': 'mask_PTV-FRONTAL-E-18GY.nii.gz'}, 'frontal_TO_mask_casca-frontal.nii.gz': {'dose': 18.0, 'rtplan_file': 'DCMRT_Plan_2.dcm', 'metastasis': 'mask_casca-frontal.nii.gz'}, 'manual_TO_mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz': {'dose': 18.0, 'rtplan_file': 'manual_input', 'metastasis': 'mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz'}, 'manual_TO_mask_PTV-soma-3x9.nii.gz': {'dose': 27.

In [122]:
files = {
    patient_id: {
        dose_file: mets
        for dose_file, mets in dose_info.items()
        if dose_file != '.ipynb_checkpoints'
    }
    for patient_id, dose_info in associated_files.items()
    if patient_id != '.ipynb_checkpoints'
}

# Optional: Print what's being loaded
pprint.pprint(files, width=120)

{'Patient_54': {'frontal_TO_mask_PTV-frontal-D-17Gy.nii.gz': {'dose': 18.0,
                                                              'metastasis': 'mask_PTV-frontal-D-17Gy.nii.gz',
                                                              'rtplan_file': 'DCMRT_Plan_1.dcm'},
                'frontal_TO_mask_PTV-frontal-E-anterior.nii.gz': {'dose': 18.0,
                                                                  'metastasis': 'mask_PTV-frontal-E-anterior.nii.gz',
                                                                  'rtplan_file': 'DCMRT_Plan_1.dcm'},
                'frontal_TO_mask_PTV-frontal-E-post-18Gy.nii.gz': {'dose': 18.0,
                                                                   'metastasis': 'mask_PTV-frontal-E-post-18Gy.nii.gz',
                                                                   'rtplan_file': 'DCMRT_Plan_1.dcm'},
                'manual_TO_mask_PTV-occiptal-E-18GY.nii.gz': {'dose': 18.0,
                                    

In [131]:
dose_distribution_files = {}

for patient_dir in os.listdir(root_dir):
    if patient_dir.startswith("."):
        continue  # skip hidden files/folders

    patient_path = os.path.join(root_dir, patient_dir)
    if not os.path.isdir(patient_path):
        continue
        
    number = patient_dir.split("_")[1]

    dicom_path = os.path.join(patient_path, f"P{number}_CT_1_TRATAMENTO")
    if not os.path.isdir(dicom_path):
        print(f"❌ Missing dicom folder for {patient_dir}: {dicom_path}")
        continue

    dose_distribution_files.setdefault(patient_dir, {})

    # List files in the dicom_path folder and filter for dose files
    for idx, file_name in enumerate(os.listdir(dicom_path)):
        file_name_upper = file_name.upper()
        if "RTDOSE" not in file_name_upper and "DCMTPS_CALCULATED" not in file_name_upper:
            continue

        rt_dose_path = os.path.join(dicom_path, file_name)

        # Define output name based on file name content
        if "DCMTPS_CALCULATED_1" in file_name_upper or "RTDOSE_1" in file_name_upper:
            output_volume_path = os.path.join(dicom_path, f"P{number}_Dose_Distribution_1.nii.gz")
        elif "DCMTPS_CALCULATED_2" in file_name_upper or "RTDOSE_2" in file_name_upper:
            output_volume_path = os.path.join(dicom_path, f"P{number}_Dose_Distribution_2.nii.gz")
        else:
            # Remove old converted files before creating new one
            for f in os.listdir(dicom_path):
                if f.endswith(".nii.gz") and "converted" in f.lower():
                    try:
                        os.remove(os.path.join(dicom_path, f))
                        print(f"Removed old converted file: {f}")
                    except Exception as e:
                        print(f"Error removing file {f}: {e}")
        
            output_volume_path = os.path.join(dicom_path, f"P{number}_Dose_Distribution.nii.gz")

        try:
            convert_rtdose(dcm_dose=rt_dose_path, force=True, dose_output_path=output_volume_path)
            print(f"✅ RTDOSE file {file_name} converted to {output_volume_path}")
            dose_distribution_files[patient_dir][output_volume_path] = file_name
        except Exception as e:
            print(f"❌ Error converting {file_name}: {e}")

✅ RTDOSE file DCMTPS_Calculated_1.dcm converted to /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_69/P69_CT_1_TRATAMENTO/P69_Dose_Distribution_1.nii.gz
Removed old converted file: DCMTPS_Calculated_converted_26.nii.gz
✅ RTDOSE file DCMTPS_Calculated.dcm converted to /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_69/P69_CT_1_TRATAMENTO/P69_Dose_Distribution.nii.gz
❌ Error converting DCMTPS_Calculated_converted_26.nii.gz: [Errno 2] No such file or directory: '/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_69/P69_CT_1_TRATAMENTO/DCMTPS_Calculated_converted_26.nii.gz'
✅ RTDOSE file DCMTPS_Calculated_1.dcm converted to /notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_89/P89_CT_1_TRATAMENTO/P89_Dose_Distribution_1.nii.gz
Removed old converted file: DCMTPS_Calculated_converted_35.nii.gz
❌ Error converting DCMTPS_Calculated_converted_35.nii.gz: [Errno 2] No such file or directory: '/notebooks/disk2/RT_Dataset_2/03_Patien

In [149]:
import os
from collections import defaultdict

def assign_dose_with_shared_rtplan(files_dict, dicom_path_template):
    result = defaultdict(dict)  # patient -> dose_file -> dict with rtplan_file + list of mets

    for patient, dose_info_dict in files_dict.items():
        patient_num = patient.split("_")[1]
        dicom_path = dicom_path_template.format(number=patient_num)

        try:
            all_nii_files = [f for f in os.listdir(dicom_path)
                             if f.endswith(".nii.gz") and "Dose_Distribution" in f]
        except Exception as e:
            print(f"⚠️ Could not list files in {dicom_path}: {e}")
            continue

        for dose_key, info in dose_info_dict.items():
            rtplan = info.get("rtplan_file", "").lower()

            # Manual input if needed
            if rtplan == "manual_input":
                print(f"\n🔧 Patient: {patient}")
                print(f"  → Metastasis: {info['metastasis']}")
                print(f"  → Dose: {info['dose']} Gy")
                manual_plan = input("    ❗ Enter RTPLAN filename (e.g. DCMRT_Plan_1.dcm): ").strip().lower()
                info["rtplan_file"] = manual_plan
                rtplan = manual_plan

            # Determine expected dose distribution file
            if "1" in rtplan:
                expected = f"P{patient_num}_Dose_Distribution_1.nii.gz"
            elif "2" in rtplan:
                expected = f"P{patient_num}_Dose_Distribution_2.nii.gz"
            else:
                expected = f"P{patient_num}_Dose_Distribution.nii.gz"

            if expected in all_nii_files:
                matched_file = expected
            else:
                print(f"⚠️ Expected file {expected} not found in {dicom_path}. Trying fallback.")
                matched_file = None
                for candidate in all_nii_files:
                    if candidate.startswith(f"P{patient_num}_Dose_Distribution"):
                        matched_file = candidate
                        break

            if matched_file:
                full_path = os.path.join(dicom_path, matched_file)

                # Initialize if not present
                if full_path not in result[patient]:
                    result[patient][full_path] = {
                        "rtplan_file": info["rtplan_file"],
                        "metastases": []
                    }

                # Append metastasis + dose
                result[patient][full_path]["metastases"].append({
                    "metastasis": info["metastasis"],
                    "dose": info["dose"]
                })
            else:
                print(f"❌ No matching dose file found for {patient} → {info['metastasis']}")

    return dict(result)

dicom_path_template = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_{number}/P{number}_CT_1_TRATAMENTO/"
final_dict = assign_dose_with_shared_rtplan(files, dicom_path_template)


🔧 Patient: Patient_89
  → Metastasis: mask_PTV-PARIETAL-ESQ-3X9GY.nii.gz
  → Dose: 18.0 Gy


    ❗ Enter RTPLAN filename (e.g. DCMRT_Plan_1.dcm):  DCMRT_Plan_1.dcm



🔧 Patient: Patient_89
  → Metastasis: mask_PTV-soma-3x9.nii.gz
  → Dose: 27.0 Gy


    ❗ Enter RTPLAN filename (e.g. DCMRT_Plan_1.dcm):  DCMRT_Plan.dcm



🔧 Patient: Patient_89
  → Metastasis: mask_PTV-LEITO-3X9GY.nii.gz
  → Dose: 27.0 Gy


...

    ❗ Enter RTPLAN filename (e.g. DCMRT_Plan_1.dcm):  DCMRT_Plan_2.dcm



🔧 Patient: Patient_85
  → Metastasis: mask_PTV-soma-3x9.nii.gz
  → Dose: 27.0 Gy


    ❗ Enter RTPLAN filename (e.g. DCMRT_Plan_1.dcm):  DCMRT_Plan_1.dcm



🔧 Patient: Patient_85
  → Metastasis: mask_PTV-3x9.nii.gz
  → Dose: 27.0 Gy


    ❗ Enter RTPLAN filename (e.g. DCMRT_Plan_1.dcm):  DCMRT_Plan_2.dcm


In [154]:
pprint(final_dict)

{'Patient_54': {'/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_54/P54_CT_1_TRATAMENTO/P54_Dose_Distribution_1.nii.gz': {'metastases': [{'dose': 18.0,
                                                                                                                                                            'metastasis': 'mask_PTV-frontal-D-17Gy.nii.gz'},
                                                                                                                                                           {'dose': 18.0,
                                                                                                                                                            'metastasis': 'mask_PTV-frontal-E-post-18Gy.nii.gz'},
                                                                                                                                                           {'dose': 18.0,
                                                                           

In [171]:
df = pd.read_excel("/notebooks/disk2/CSV_Dataset/Volumes_Doses.xlsx")  # ou .csv com read_csv
#print(df.head())
patients = ["Patient_68"]

for patient, data in final_dict.items():
    print(f"Processing {patient}")
    number = str(patient).replace("Patient_", "").replace("P", "")
    documents = final_dict[patient]
    if patient in patient_masks_dose_more:
        for dose_distribution_file, info in documents.items():
            #print(f" * This is the {dose_distribution_file} for {patient} *")
            dose_distribution = nib.load(dose_distribution_file)
            dose_distribution_data = dose_distribution.get_fdata()
            rtplan_file = info['rtplan_file']
            metastases_list = info['metastases']

            for met in metastases_list:
                metastasis_filename = met['metastasis']
                dose_value = met['dose']

                print(f" → Metastasis file: {metastasis_filename}")
                print(f" → Dose: {dose_value} Gy")

                
                mask_name = os.path.basename(metastasis_filename).replace(".nii.gz", "")
                matched_rows = df[(df["File Name"] == mask_name) & (df["Patient ID"] == patient)]

                if not matched_rows.empty:
                    for _, row in matched_rows.iterrows():
                        if patient in patients: # the dose description from the DICOM files was used 
                            prescription_dose = row["Dose Media"]
                            print(f"  * Prescription dose for {patient} and mask {mask_name}: {prescription_dose}")
                            isodose_volume_mask = (dose_distribution_data >= prescription_dose)
                            
                           
                        else: #description used from the Dicomplyer
                            prescription_dose = row["Dose Media"] / 100
                            print(f"   * Prescription dose for {patient} and mask {mask_name}: {prescription_dose}")
                            isodose_volume_mask = (dose_distribution_data >= prescription_dose)
                
                        isodose_volume_image = nib.Nifti1Image(isodose_volume_mask, dose_distribution.affine, dose_distribution.header)
                        
                        isodose_volume_path = f"/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_{number}/P{number}_CT_1_TRATAMENTO/P{number}_Isodose_Volume_{metastasis_filename}"
                        nib.save(isodose_volume_image, isodose_volume_path)
                        print(f" Isodose mask saved for {metastasis_filename}")
                else:
                    
                    print(f" No matching dose found for {patient} and mask {mask_name}, so will be using the {dose_value}")
                    prescription_dose = dose_value
                    isodose_volume_mask = (dose_distribution_data >= prescription_dose)
                    isodose_volume_path = f"/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_{number}/P{number}_CT_1_TRATAMENTO/P{number}_Isodose_Volume_{metastasis_filename}"
                    nib.save(isodose_volume_image, isodose_volume_path)
                    print(f" Isodose mask saved for {metastasis_filename}")
                    

                # Resize Isodose Volume Mask to the same caracteristics as the original mask volume
                path = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_" + number + "/P" + number + "_CT_1_TRATAMENTO/"
                tumor_mask_path = os.path.join(path, metastasis_filename)
                # Check if tumor mask exists
                if not os.path.exists(tumor_mask_path):
                    print(f" ⚠️ Tumor mask file does not exist: {tumor_mask_path}, skipping.")
                    continue
                
                tumor_mask = sitk.ReadImage(tumor_mask_path) # tumor path
                isodose_volume = sitk.ReadImage(isodose_volume_path) # isodose path
        
                resampler = sitk.ResampleImageFilter()
        
                resampler.SetReferenceImage(tumor_mask)
            
                resampler.SetInterpolator(sitk.sitkNearestNeighbor)
                
                resampler.SetOutputOrigin(tumor_mask.GetOrigin())
                resampler.SetOutputSpacing(tumor_mask.GetSpacing())
                resampler.SetOutputDirection(tumor_mask.GetDirection())
                resampler.SetSize(tumor_mask.GetSize())
                
                resampled_volume = resampler.Execute(isodose_volume)
        
                resampled_volume_path = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_" + number + "/P" + number + "_CT_1_TRATAMENTO/P" + number + "_Isodose_Volume_" + mask_name + "_Resampled.nii.gz"
                sitk.WriteImage(resampled_volume, resampled_volume_path)
                print(f"Isodose Prescription Volume resampled")
    
                # Filter for each mask
                isodose_volume_resized = nib.load(resampled_volume_path)
                isodose_volume_resized_data = isodose_volume_resized.get_fdata()
        
                mask_tumor = nib.load(tumor_mask_path)
                mask_tumor_data = mask_tumor.get_fdata()
                
                coords = np.argwhere(mask_tumor_data > 0)
                centroide = np.mean(coords, axis=0).astype(int)
        
                shape = isodose_volume_resized_data.shape
                x, y, z = np.ogrid[:shape[0], :shape[1], :shape[2]]
                
                dx, dy, dz = mask_tumor.header.get_zooms() #so the distance is in mm
                distance = np.sqrt(
                    ((x - centroide[0]) * dx) ** 2 +
                    ((y - centroide[1]) * dy) ** 2 +
                    ((z - centroide[2]) * dz) ** 2
                )
                
                standard_volume_cm3 = 1.0
                
                if not matched_rows.empty:
                    volume_cm3 = matched_rows.iloc[0]["Volume"]
                else:
                    volume_cm3 = standard_volume_cm3
                    print(f" ⚠️ Using default volume of {standard_volume_cm3} cm³ for patient {patient}, mask {metastasis_filename}")
                    
                volume_mm3 = volume_cm3 * 1000
                r = (3 * volume_mm3 / (4 * np.pi)) ** (1/3)
                r_new = (3 * 1.50 * volume_mm3 / (4 * np.pi)) ** (1/3) #margem de 50% mais para o número de voxeis ser aceitável
                margin = r_new - r
                min_voxel_size = min(dx, dy, dz)  # em mm
                if margin < min_voxel_size: # isto nos casos em que a margin é menor do que 1, o que significa que temos que garantir que a margem seja pelo menos igual ao tamanho de um voxel
                    print(f"Margin smaller than 1 voxel ({margin:.2f} mm), adjusting to {min_voxel_size:.2f} mm")
                    margin = min_voxel_size
                print(f"Computed margin: {margin:.2f} mm for patient {patient}, mask {mask_name}")

                sphere_mask = distance <= margin
                print(f"Voxels in sphere_mask:", np.count_nonzero(sphere_mask))
        
                filtered_volume = isodose_volume_resized_data * sphere_mask
                final_volume = nib.Nifti1Image(filtered_volume, affine=isodose_volume_resized.affine)
        
                filtered_volume_path = "/notebooks/disk2/RT_Dataset_2/03_Patients_NifTI_Dose_More/Patient_" + number + "/P" + number + "_CT_1_TRATAMENTO/P" + number + "_Isodose_Volume_" + mask_name + "_Resampled_Filtered.nii.gz"
                nib.save(final_volume, filtered_volume_path)
                print(f"Isodose Prescription Volume Filtered")
                print()
           

Processing Patient_69
 → Metastasis file: mask_PTV-temporal-1X20GY.nii.gz
 → Dose: 20.0 Gy
   * Prescription dose for Patient_69 and mask mask_PTV-temporal-1X20GY: 1.08992
 Isodose mask saved for mask_PTV-temporal-1X20GY.nii.gz
Isodose Prescription Volume resampled
Margin smaller than 1 voxel (0.53 mm), adjusting to 0.79 mm
Computed margin: 0.79 mm for patient Patient_69, mask mask_PTV-temporal-1X20GY
Voxels in sphere_mask: 5
Isodose Prescription Volume Filtered

 → Metastasis file: mask_PTV-leito-5x6Gy.nii.gz
 → Dose: 30.0 Gy
   * Prescription dose for Patient_69 and mask mask_PTV-leito-5x6Gy: 0.0025800000000000003
 Isodose mask saved for mask_PTV-leito-5x6Gy.nii.gz
Isodose Prescription Volume resampled
Computed margin: 3.30 mm for patient Patient_69, mask mask_PTV-leito-5x6Gy
Voxels in sphere_mask: 239
Isodose Prescription Volume Filtered

Processing Patient_89
 → Metastasis file: mask_PTV-TEMPORAL-ESQ-18GY.nii.gz
 → Dose: 18.0 Gy
   * Prescription dose for Patient_89 and mask mask_P